In [1]:
import os
import sys

# External imports and Qiskit imports
from qiskit import Aer, IBMQ
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.visualization import circuit_drawer
from qiskit.algorithms.optimizers import SPSA
from qiskit.providers.ibmq.runtime import UserMessenger
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Put qiskit-machine-learning subtree at front of Python path
ml_path = os.path.abspath(os.path.join('../qiskit-machine-learning'))
sys.path.insert(0, ml_path)
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Put this repository on the Python path and import qkt pkgs
module_path = os.path.abspath(os.path.join('../'))
sys.path.append(module_path)
from qkt.feature_maps import CovariantFeatureMap
from qkt.utils import train_test_split, QKTCallback

In [2]:
provider = IBMQ.load_account()

In [3]:
#program_id = provider.runtime.upload_program(
#    data="../../ntc-ibm-programs/runtime/quantum_kernel_trainer.py",
#    metadata="../../ntc-ibm-programs/runtime/quantum_kernel_trainer.json")
#print(program_id)

In [4]:
#print(provider.runtime.delete_program('quantum-kernel-trainer-E9m4wwJ1KB'))

In [5]:
# Load the dataset and split into train and test sets
DATA_FILEPATH = '../data/dataset_graph7.csv'
X_train, y_train, X_test, y_test = train_test_split(DATA_FILEPATH)

In [6]:
num_features = np.shape(X_train)[1]
entangler_map = [[0,2],[3,4],[2,5],[1,4],[2,3],[4,6]]

# Note that [[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]]
# is a suitable input for the 10-qubit dataset

fm = CovariantFeatureMap(
    feature_dimension=num_features,
    entanglement=entangler_map,
    single_training_parameter=True
)
circuit_drawer(fm)

┌──────────────┐             ░ ┌─────────────────┐ ┌─────────────────┐ 
q_0: ┤ Ry(θ_par[0]) ├─■───────────░─┤ Rz(-2*x_par[1]) ├─┤ Rx(-2*x_par[0]) ├─
     ├──────────────┤ │           ░ ├─────────────────┤ ├─────────────────┤ 
q_1: ┤ Ry(θ_par[0]) ├─┼─────■─────░─┤ Rz(-2*x_par[3]) ├─┤ Rx(-2*x_par[2]) ├─
     ├──────────────┤ │     │     ░ ├─────────────────┤ ├─────────────────┤ 
q_2: ┤ Ry(θ_par[0]) ├─■──■──┼──■──░─┤ Rz(-2*x_par[5]) ├─┤ Rx(-2*x_par[4]) ├─
     ├──────────────┤    │  │  │  ░ ├─────────────────┤ ├─────────────────┤ 
q_3: ┤ Ry(θ_par[0]) ├─■──┼──┼──■──░─┤ Rz(-2*x_par[7]) ├─┤ Rx(-2*x_par[6]) ├─
     ├──────────────┤ │  │  │     ░ ├─────────────────┤ ├─────────────────┤ 
q_4: ┤ Ry(θ_par[0]) ├─■──┼──■──■──░─┤ Rz(-2*x_par[9]) ├─┤ Rx(-2*x_par[8]) ├─
     ├──────────────┤    │     │  ░ ├─────────────────┴┐├─────────────────┴┐
q_5: ┤ Ry(θ_par[0]) ├────■─────┼──░─┤ Rz(-2*x_par[11]) ├┤ Rx(-2*x_par[10]) ├
     ├──────────────┤          │  ░ ├──────────────────┤├──────────────────┤
q_6: ┤ Ry(θ_par[0]) ├──────────■──░─┤ Rz(-2*x_par[13]) ├┤ Rx(-2*x_par[12]) ├
     └──────────────┘             ░ └──────────────────┘└──────────────────┘

In [7]:
# Use the qasm simulator backend
backend = provider.get_backend('simulator_statevector')

# Instantiate quantum kernel
quant_kernel = QuantumKernel(fm,
                             user_parameters=fm.user_parameters,
                             quantum_instance=backend)

# Set up the optimizer
spsa_opt = SPSA(maxiter=100, learning_rate=0.1, perturbation=0.1)

runtime_inputs = {
    "quantum_kernel": quant_kernel,
    "data": X_train,
    "labels": y_train,
    "optimizer": spsa_opt,
    "shots": 1,
    "initial_point": [0.1],
    "measurement_error_mitigation": False,
}

options = {'backend_name': 'simulator_statevector'}

job = provider.runtime.run(program_id="quantum-kernel-trainer-39vqXArNdB",
                           options=options,
                           image="quantum-kernel-trainer:latest",
                           inputs=runtime_inputs
                          )

print(job.job_id())
final_results = job.result()
print(f"final result: {final_results}")

c6khf1q01r3clbe7igl0
final result: {'optimizer_evals': 200, 'optimal_value': 10.172856806272426, 'optimal_point': array([0.320603])}


In [8]:
quant_kernel.assign_user_parameters(final_results['optimal_point'])

In [9]:
# Use QSVC for classification
qsvc = QSVC(quantum_kernel=quant_kernel)

# Fit the QSVC
qsvc.fit(X_train, y_train)

# Predict the labels
labels_test = qsvc.predict(X_test)

# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

accuracy test: 0.75
